In [ ]:
!pip install -q fastapi uvicorn pyngrok nest_asyncio jinja2 \
               langchain langchain-community langchain-groq chromadb pypdf


In [ ]:
import os
from pyngrok import ngrok

os.environ["GROQ_API_KEY"] = "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"
NGROK_AUTH_TOKEN = "316pNuAwhjYD8giBKQblyXolHAf_6r1FYADkHEz72xGogbbik"


In [ ]:
!ngrok config add-authtoken $NGROK_AUTH_TOKEN


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
code = r'''
import os
from typing import List, Optional

from fastapi import FastAPI, Request, Form, UploadFile, File
from fastapi.responses import HTMLResponse, RedirectResponse
from fastapi.templating import Jinja2Templates

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# ---------- Paths & Globals ----------
UPLOAD_DIR = "/content/uploads"
DB_DIR = "/content/chroma_db"
os.makedirs(UPLOAD_DIR, exist_ok=True)
os.makedirs(DB_DIR, exist_ok=True)

templates = Jinja2Templates(directory="templates")
app = FastAPI()

# Chat state
CHAT_HISTORY: List[tuple] = []
SYSTEM_PROMPT_DEFAULT = (
    "You are a helpful assistant that answers ONLY using information from the provided documents. "
    "If you don’t find the answer in the documents, say you don’t know and ask the user to upload "
    "more relevant PDFs. Keep answers concise and in English."
)
current_system_prompt = SYSTEM_PROMPT_DEFAULT

# LLM (Groq)
llm = ChatGroq(
    api_key=os.environ.get("GROQ_API_KEY"),
    model="llama3-8b-8192",
    temperature=0.3
)

# Vector DB / QA chain
vectordb: Optional[Chroma] = None
qa_chain: Optional[RetrievalQA] = None


# ---------- Helper to (re)build the vector DB + chain ----------
def rebuild_index_and_chain():
    global vectordb, qa_chain, current_system_prompt

    # Load PDFs
    docs = []
    for fname in os.listdir(UPLOAD_DIR):
        if fname.lower().endswith(".pdf"):
            loader = PyPDFLoader(os.path.join(UPLOAD_DIR, fname))
            docs.extend(loader.load())

    if not docs:
        vectordb = None
        qa_chain = None
        return

    # Split
    splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=120)
    chunks = splitter.split_documents(docs)

    # Embed + index
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectordb = Chroma.from_documents(
        chunks,
        embedding=embeddings,
        persist_directory=DB_DIR
    )
    vectordb.persist()

    # Custom prompt (system_prompt gets injected directly here)
    prompt_text = (
        f"{current_system_prompt}\n\n"
        "Context from documents:\n{context}\n\n"
        "Question: {question}\n\n"
        "Answer in English:\n"
    )
    prompt = PromptTemplate(
        input_variables=["context", "question"],
        template=prompt_text
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=vectordb.as_retriever(search_kwargs={"k": 4}),
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=False
    )


# ---------- Routes ----------
@app.get("/", response_class=HTMLResponse)
async def upload_page(request: Request):
    files = [f for f in os.listdir(UPLOAD_DIR) if f.lower().endswith(".pdf")]
    return templates.TemplateResponse("upload.html", {"request": request, "files": files})


@app.post("/upload", response_class=RedirectResponse)
async def upload_files(files: List[UploadFile] = File(...)):
    for file in files:
        if not file.filename.lower().endswith(".pdf"):
            continue
        out_path = os.path.join(UPLOAD_DIR, file.filename)
        with open(out_path, "wb") as f:
            f.write(await file.read())
    rebuild_index_and_chain()
    return RedirectResponse(url="/chat", status_code=303)


@app.post("/reindex", response_class=RedirectResponse)
async def reindex():
    rebuild_index_and_chain()
    return RedirectResponse(url="/chat", status_code=303)


@app.get("/chat", response_class=HTMLResponse)
async def chat_page(request: Request):
    return templates.TemplateResponse("chat.html", {
        "request": request,
        "chat_history": CHAT_HISTORY,
        "system_prompt": current_system_prompt
    })


@app.post("/chat", response_class=HTMLResponse)
async def chat(request: Request, message: str = Form(...), prompt: str = Form(None)):
    global current_system_prompt, qa_chain

    # Update system prompt (and rebuild prompt inside chain)
    if prompt and prompt.strip():
        current_system_prompt = prompt.strip()
        rebuild_index_and_chain()

    if qa_chain is None:
        bot = "⚠️ No PDFs indexed yet. Upload PDFs first."
        CHAT_HISTORY.append(("You", message))
        CHAT_HISTORY.append(("Bot", bot))
        return templates.TemplateResponse("chat.html", {
            "request": request,
            "chat_history": CHAT_HISTORY,
            "system_prompt": current_system_prompt
        })

    # Run QA
    result = qa_chain.invoke({"query": message})
    answer = result.get("result", "⚠️ I couldn’t generate an answer.")

    CHAT_HISTORY.append(("You", message))
    CHAT_HISTORY.append(("Bot", answer))

    return templates.TemplateResponse("chat.html", {
        "request": request,
        "chat_history": CHAT_HISTORY,
        "system_prompt": current_system_prompt
    })
'''
with open("main.py", "w") as f:
    f.write(code)
print("✅ Wrote main.py")


✅ Wrote main.py


In [ ]:
import os

os.makedirs("templates", exist_ok=True)

upload_html = r"""
<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8"/>
  <title>Upload PDFs</title>
  <style>
    body { font-family: Arial, sans-serif; max-width: 860px; margin: 30px auto; }
    .card { background: #fff; border: 1px solid #eee; border-radius: 12px; padding: 16px; margin-bottom: 16px; }
    button { padding: 8px 14px; border-radius: 10px; border: 1px solid #ddd; background: #f0f0f0; cursor: pointer; }
    input[type=file] { padding: 8px; }
    ul { line-height: 1.8; }
    a { text-decoration: none; }
  </style>
</head>
<body>
  <h1>Upload PDFs for RAG</h1>

  <div class="card">
    <form action="/upload" method="post" enctype="multipart/form-data">
      <input type="file" name="files" accept=".pdf" multiple>
      <button type="submit">Upload & Index</button>
    </form>
  </div>

  <div class="card">
    <h3>Uploaded PDFs</h3>
    <ul>
      {% for f in files %}
        <li>{{ f }}</li>
      {% endfor %}
      {% if files|length == 0 %}
        <li><i>No PDFs uploaded yet.</i></li>
      {% endif %}
    </ul>
  </div>

  <div class="card">
    <form action="/reindex" method="post">
      <button type="submit">Rebuild Index</button>
      <a href="/chat"><button type="button">Go to Chat</button></a>
    </form>
  </div>
</body>
</html>
"""

chat_html = r"""
<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8"/>
  <title>Chat with your PDFs</title>
  <style>
    body { font-family: Arial, sans-serif; max-width: 860px; margin: 30px auto; }
    .card { background: #fff; border: 1px solid #eee; border-radius: 12px; padding: 16px; margin-bottom: 16px; }
    .chat-box { background: #fafafa; border: 1px solid #eee; border-radius: 12px; padding: 14px; min-height: 260px; }
    .you  { color: #1b6; }
    .bot  { color: #06c; }
    textarea, input[type=text] { width: 100%; padding: 10px; border-radius: 10px; border: 1px solid #ddd; }
    button { padding: 8px 14px; border-radius: 10px; border: 1px solid #ddd; background: #f0f0f0; cursor: pointer; }
    .row { display: flex; gap: 12px; align-items: center; }
  </style>
</head>
<body>
  <h1>Chat with your PDFs</h1>
  <p><a href="/">⬅️ Upload more PDFs</a></p>

  <div class="card">
    <h3>System Prompt (Persona/Rules)</h3>
    <form method="post" action="/chat">
      <textarea name="prompt" rows="5">{{ system_prompt }}</textarea>
      <p><small>Tip: This changes how the model behaves. Keep it concise.</small></p>
      <input type="text" name="message" placeholder="Type your question about the uploaded PDFs..."/>
      <div class="row">
        <button type="submit">Send</button>
        <a href="/"><button type="button">Upload PDFs</button></a>
      </div>
    </form>
  </div>

  <div class="card chat-box">
    {% for sender, text in chat_history %}
      <p class="{{ 'you' if sender == 'You' else 'bot' }}"><b>{{ sender }}:</b> {{ text }}</p>
    {% endfor %}
    {% if chat_history|length == 0 %}
      <p><i>No messages yet. Ask something!</i></p>
    {% endif %}
  </div>
</body>
</html>
"""

with open("templates/upload.html", "w") as f:
    f.write(upload_html)

with open("templates/chat.html", "w") as f:
    f.write(chat_html)

print("✅ Wrote templates/upload.html and templates/chat.html")


✅ Wrote templates/upload.html and templates/chat.html


In [ ]:
import nest_asyncio
from pyngrok import ngrok

nest_asyncio.apply()

# Start tunnel
public_url = ngrok.connect(8000)
print("🔗 Public URL:", public_url)

# Start Uvicorn (serves main:app)
!uvicorn main:app --host 0.0.0.0 --port 8000 --reload


🔗 Public URL: NgrokTunnel: "https://b597e445a051.ngrok-free.app" -> "http://localhost:8000"
INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [13591] using WatchFiles
INFO:     Started server process [13593]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     197.58.238.139:0 - "GET / HTTP/1.1" 200 OK
INFO:     197.58.238.139:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
/content/main.py:69: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentenc